In [27]:
import pandas as pd

# Load CSVs into dataframes
dataframes = [pd.read_csv(csv_file) for csv_file in csv_files]

# Example: Viewing the first few rows of the first dataframe
print(dataframes[0].head())


NameError: name 'csv_files' is not defined

In [28]:
from PIL import Image
import os

# Directory where the extracted images are stored
extract_folder = r"C:\Users\HP\Desktop\Image Recognitation\extracted_data"

# Target directory to save processed images
processed_dir = r"C:\Users\HP\Desktop\Image Recognitation\processed_images"
os.makedirs(processed_dir, exist_ok=True)

# Loop through each subdirectory in the extract_folder
for subdir in os.listdir(extract_folder):
    subdir_path = os.path.join(extract_folder, subdir)

    # Process only directories
    if os.path.isdir(subdir_path):
        for img_name in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_name)

            # Only process files
            if os.path.isfile(img_path):
                try:
                    with Image.open(img_path) as img:
                        # Example preprocessing: resizing the image to (224, 224)
                        img_resized = img.resize((224, 224))
                        # Save the preprocessed image
                        img_resized.save(os.path.join(processed_dir, img_name))
                except Exception as e:
                    print(f"Failed to process {img_name}: {e}")

print("Image preprocessing complete.")


Image preprocessing complete.


In [29]:
from PIL import Image
import os
import numpy as np

# Directory where the extracted images are stored
extract_folder = r"C:\Users\HP\Desktop\Image Recognitation\extracted_data"

# Target directory to save processed images
processed_dir = r"C:\Users\HP\Desktop\Image Recognitation\processed_images"
os.makedirs(processed_dir, exist_ok=True)

# Loop through each subdirectory in the extract_folder
for subdir in os.listdir(extract_folder):
    subdir_path = os.path.join(extract_folder, subdir)

    # Process only directories
    if os.path.isdir(subdir_path):
        for img_name in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_name)

            # Only process files
            if os.path.isfile(img_path):
                try:
                    with Image.open(img_path) as img:
                        # Resize the image to (224, 224)
                        img_resized = img.resize((224, 224))
                        
                        # Convert image to a NumPy array and normalize
                        img_array = np.array(img_resized).astype(np.float32) / 255.0
                        
                        # Convert back to an image
                        img_normalized = Image.fromarray((img_array * 255).astype(np.uint8))
                        
                        # Save the preprocessed image
                        img_normalized.save(os.path.join(processed_dir, img_name))
                except Exception as e:
                    print(f"Failed to process {img_name}: {e}")

print("Image preprocessing and normalization complete.")


Image preprocessing and normalization complete.


In [30]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import numpy as np

# Define the directory where the images are processed
processed_dir = r"C:\Users\HP\Desktop\Image Recognitation\processed_images"

# Directory to save extracted features
features_dir = r"C:\Users\HP\Desktop\Image Recognitation\features"
os.makedirs(features_dir, exist_ok=True)

# Define the image transformation to match the input of ResNet18
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Function to extract features from an image using the pre-trained model
def extract_features(image_path, model, transform):
    # Load and transform the image
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
    
    with torch.no_grad():
        features = model(img_tensor)
    
    return features.squeeze().numpy()  # Remove batch dimension and convert to numpy array

# Process each image and extract features
for img_name in os.listdir(processed_dir):
    img_path = os.path.join(processed_dir, img_name)
    
    if os.path.isfile(img_path):
        try:
            features = extract_features(img_path, model, transform)
            np.save(os.path.join(features_dir, img_name.replace('.jpg', '.npy')), features)
        except Exception as e:
            print(f"Failed to extract features from {img_name}: {e}")

print("Feature extraction complete.")


Feature extraction complete.


In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
df = pd.read_csv('questions_responses.csv')

# Extract questions
queries = df['Question'].values

# Tokenization and encoding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(queries)
sequences = tokenizer.texts_to_sequences(queries)

# Padding sequences to ensure uniform input size
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Print the results
print("Tokenized and padded queries:")
print(padded_sequences)

# Optionally save tokenizer for later use
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


FileNotFoundError: [Errno 2] No such file or directory: 'questions_responses.csv'

In [32]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Assuming `padded_sequences` is your tokenized and padded text data
def normalize_sequences(sequences):
    # Convert sequences to numpy array for normalization
    sequences_np = np.array(sequences, dtype=float)
    
    # Initialize the StandardScaler
    scaler = StandardScaler()
    
    # Normalize the sequences
    normalized_sequences = scaler.fit_transform(sequences_np)
    
    return normalized_sequences

# Example usage
normalized_sequences = normalize_sequences(padded_sequences)
print("Normalized tokenized and padded queries:")
print(normalized_sequences)


NameError: name 'padded_sequences' is not defined

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Parameters
vocab_size = 10000  # Size of the tokenizer vocabulary
embedding_dim = 128  # Dimension of embedding space
lstm_units = 64  # Number of LSTM units
input_length = padded_sequences.shape[1]  # Length of padded sequences
num_classes = 10     # Number of classes for classification (adjust this as needed)

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length),
    LSTM(lstm_units, return_sequences=False),
    Dense(lstm_units, activation='relu')
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model (for demonstration, you would need proper training data and labels)
# model.fit(padded_sequences, labels, epochs=10, batch_size=32)

# Extract features
text_features = model.predict(padded_sequences)
print("Extracted features:")
print(features)


NameError: name 'padded_sequences' is not defined

In [34]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assuming `query_features` is your text feature matrix
scaler = StandardScaler()
normalized_query_features = scaler.fit_transform(features)


ValueError: Expected 2D array, got 1D array instead:
array=[-1.9939196e+00 -8.4555626e-01 -2.3387229e+00 -4.4125257e+00
 -3.1227643e+00  2.4605188e+00  1.0050197e+00 -6.3310319e-01
  7.2966111e-01  3.8137548e+00  7.4052100e+00  3.9826109e+00
  2.4990387e+00  6.6070523e+00  3.0617247e+00  8.6111641e+00
  5.5946555e+00  4.8946190e+00  3.9981496e+00  7.3557539e+00
  1.0201323e+01  4.5370078e+00  3.3402932e+00  6.7482436e-01
  3.9036582e+00  2.3915422e+00  6.6188736e+00  3.5156238e+00
 -8.9494079e-02 -1.3304713e+00  3.3425145e+00  6.2310925e+00
  4.9661999e+00  1.2399760e+00  1.9878036e+00  3.7653320e+00
  4.0677128e+00  2.1187885e+00  5.5738254e+00  8.5208273e+00
  1.0867703e+01  8.4217033e+00  1.0075059e+01  9.4817476e+00
  7.7223659e+00  7.3388678e-01  8.7565298e+00  7.2358384e+00
  2.9037430e+00  1.9532803e+00  1.7272254e+00 -1.6512737e+00
  4.5064087e+00  5.0155101e+00  5.4110885e+00  6.6788139e+00
  1.5066624e+00  5.3771424e+00  6.7508063e+00  1.0033520e+01
  6.9514098e+00  2.8652630

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the last time step
        return out

# Example instantiation and usage
lstm_model = LSTMModel(input_size=embedding_dim, hidden_size=lstm_units, output_size=num_classes)


In [ ]:
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def extract_text_features(text):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()  # Mean pooling


In [ ]:
print(f"combined_features shape: {combined_features.shape}")
print(f"labels shape: {labels.shape}")


In [ ]:
import numpy as np

num_samples = combined_features.shape[0]
num_classes = 5  # Number of categories, adjust as needed

# Create labels with the same number of samples as combined_features
labels = np.random.randint(0, num_classes, num_samples)

print(f"Updated labels shape: {labels.shape}")


In [35]:
from sklearn.linear_model import LogisticRegression

# Assuming `image_features` and `text_features` are numpy arrays
combined_features = np.concatenate((features, text_features), axis=1)

# Train a classifier
classifier = LogisticRegression()
classifier.fit(combined_features, labels)  # `labels` should be your target variable

# Test the classifier
predictions = classifier.predict(combined_features)


AxisError: axis 1 is out of bounds for array of dimension 1

In [36]:
from sklearn.linear_model import LogisticRegression

# Assuming `image_features` and `text_features` are numpy arrays
combined_features = np.concatenate((features, text_features), axis=1)

# Train a classifier
classifier = LogisticRegression()
classifier.fit(combined_features, labels)  # `labels` should be your target variable

# Test the classifier
predictions = classifier.predict(combined_features)


AxisError: axis 1 is out of bounds for array of dimension 1

In [37]:
from sklearn.decomposition import PCA

# Reduce dimensionality
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(combined_features)


ValueError: n_components=100 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [ ]:
# Assuming you need to select a subset of combined_features
subset_size = labels.shape[0]
combined_features_subset = combined_features[:subset_size]

print(f"Subset combined_features shape: {combined_features_subset.shape}")


In [38]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertModel
from PIL import Image
import torchvision.transforms as transforms
from torchvision import models

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained models
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
cnn_model = models.resnet18(pretrained=True).to(device)
cnn_model.eval()

# Image preprocessing
def preprocess_image(img_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = cnn_model(img_tensor)
    return features.squeeze().cpu().numpy()

# Text feature extraction
def extract_text_features(text):
    inputs = tokenizer(text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()


C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weig

In [39]:
# Example data
image_path = r"C:\Users\HP\Desktop\Image Recognitation\images\002_0087.jpg"
text_query = 'What\'s the color?'

# Extract features
image_features = preprocess_image(image_path)
text_features = extract_text_features(text_query)

# Ensure they are 2D
image_features = image_features.reshape(1, -1)
text_features = text_features.reshape(1, -1)

# Combine features
combined_features = np.concatenate((image_features, text_features), axis=1)

print("Combined features shape:", combined_features.shape)


Combined features shape: (1, 1768)


In [40]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Simulated data
num_samples = 100  # Sample size
num_features_image = 512  # Example number of image features
num_features_text = 768  # Example number of text features
num_classes = 5

# Generate dummy data
np.random.seed(102)
combined_features = np.random.rand(num_samples, num_features_image + num_features_text)
labels = np.random.randint(0, num_classes, num_samples)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.3, random_state=42)

# Normalization
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Dimensionality Reduction
pca = PCA(n_components=min(70, X_train_normalized.shape[1]))  # Use 70 or fewer components
X_train_reduced = pca.fit_transform(X_train_normalized)
X_test_reduced = pca.transform(X_test_normalized)

# Train the classifier
classifier = LogisticRegression(max_iter=1000)  # Increased max_iter to ensure convergence
classifier.fit(X_train_reduced, y_train)

# Predict and evaluate
predictions = classifier.predict(X_test_reduced)
accuracy = accuracy_score(y_test, predictions)

print(f"Model Accuracy: {accuracy:.2f}")


Model Accuracy: 0.40


In [41]:
# Define feature extraction functions (placeholders)
def preprocess_image(image_path):
    # Implement image preprocessing and feature extraction
    return np.random.rand(num_features_image)

def extract_text_features(text_query):
    # Implement text feature extraction
    return np.random.rand(num_features_text)

# Define the model response function
def get_model_response(image_path, text_query):
    # Extract features
    image_features = preprocess_image(image_path)
    text_features = extract_text_features(text_query)
    
    # Ensure the features are 2D arrays
    image_features = image_features.reshape(1, -1)  # Convert to 2D array with 1 row
    text_features = text_features.reshape(1, -1)  # Convert to 2D array with 1 row
    
    # Combine features
    combined_features = np.concatenate((image_features, text_features), axis=1)
    
    # Normalize and reduce dimensions
    normalized_features = scaler.transform(combined_features)  # Use transform
    reduced_features = pca.transform(normalized_features)  # Use transform
    
    # Classification
    prediction = classifier.predict(reduced_features)
    return prediction



In [42]:
# Example usage
image_path = r"C:\Users\HP\Desktop\Image Recognitation\images\002_0087.jpg"
text_query = 'Whats the color?'
response = get_model_response(image_path, text_query)

print(f"Model Response: {response}")

Model Response: [3]


In [43]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

# Load pre-trained text generation model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

def generate_description(features):
    # Create a prompt based on features
    prompt = f"Describe the image with features: {features}"
    
    # Tokenize the prompt with a large enough max_length
    inputs = tokenizer(prompt, return_tensors='pt', max_length=1024, truncation=True)
    
    # Generate description
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs.get('attention_mask'),
        max_length=150,  # Adjust this to ensure it does not exceed model capacity
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=150,  # This parameter helps control the length of the generated output
        do_sample=True  # Optional: Enable sampling to get more varied responses
    )
    
    # Decode the generated text
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

# Example usage
image_path = r"C:\Users\HP\Desktop\Image Recognitation\images\002_0087.jpg"
text_query = 'What is in the picture?'
response = get_model_response(image_path, text_query)

print(f"Model Response: {response}")


C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model Response: [1]


In [44]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from PIL import Image

# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def generate_description(image_path):
    # Load and process the image
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    # Generate a description
    outputs = model.generate(**inputs)
    
    # Decode the description
    description = processor.decode(outputs[0], skip_special_tokens=True)
    return description


C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [45]:

# Example usage
image_path = r"C:\Users\HP\Desktop\Image Recognitation\images\Lincoln_Sparrow_0023_2887691144.jpg"
description = generate_description(image_path)
print(f"Model Description: {description}")


C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Model Description: a bird with a yellow beak


In [47]:
#!/usr/bin/env python
# coding: utf-8

import os
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2LMHeadModel, BlipProcessor, BlipForConditionalGeneration

# Setup paths
extract_folder = r"C:\Users\HP\Desktop\Image Recognitation\extracted_data"
processed_dir = r"C:\Users\HP\Desktop\Image Recognitation\processed_images"
features_dir = r"C:\Users\HP\Desktop\Image Recognitation\features"
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(features_dir, exist_ok=True)

# Image preprocessing function
def preprocess_image(img_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        features = cnn_model(img_tensor).squeeze().cpu().numpy()
    return features

# Define feature extraction for text
def extract_text_features(text):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Load models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_model = models.resnet18(pretrained=True).to(device)
cnn_model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

# Process images and extract features
for subdir in os.listdir(extract_folder):
    subdir_path = os.path.join(extract_folder, subdir)
    if os.path.isdir(subdir_path):
        for img_name in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_name)
            if os.path.isfile(img_path):
                try:
                    features = preprocess_image(img_path)
                    np.save(os.path.join(features_dir, img_name.replace('.jpg', '.npy')), features)
                except Exception as e:
                    print(f"Failed to process {img_name}: {e}")

print("Image preprocessing and feature extraction complete.")

# Load question-response data
df = pd.read_csv(r'C:\Users\HP\Desktop\SIH\questions_responses.csv')
queries = df['Question'].values

# Tokenization and padding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(queries)
sequences = tokenizer.texts_to_sequences(queries)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Normalize sequences
def normalize_sequences(sequences):
    sequences_np = np.array(sequences, dtype=float)
    scaler = StandardScaler()
    return scaler.fit_transform(sequences_np)

normalized_sequences = normalize_sequences(padded_sequences)

# Define and train the LSTM model
embedding_dim = 128
lstm_units = 64
num_classes = 10
model = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_length),
    LSTM(lstm_units, return_sequences=False),
    Dense(lstm_units, activation='relu')
])
model.compile(optimizer='adam', loss='mean_squared_error')
# model.fit(normalized_sequences, labels, epochs=10, batch_size=32)  # Assuming labels are defined

# Combine features for classification
image_features = np.random.rand(100, 512)  # Placeholder
text_features = model.predict(normalized_sequences)
combined_features = np.concatenate((image_features, text_features), axis=1)

# Normalize and reduce dimensions
scaler = StandardScaler()
X_normalized = scaler.fit_transform(combined_features)
pca = PCA(n_components=100)
X_reduced = pca.fit_transform(X_normalized)

# Train and evaluate the classifier
labels = np.random.randint(0, num_classes, combined_features.shape[0])
X_train, X_test, y_train, y_test = train_test_split(X_reduced, labels, test_size=0.3, random_state=42)
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

# Define feature extraction and response generation
def get_model_response(image_path, text_query):
    image_features = preprocess_image(image_path).reshape(1, -1)
    text_features = extract_text_features(text_query).reshape(1, -1)
    combined_features = np.concatenate((image_features, text_features), axis=1)
    normalized_features = scaler.transform(combined_features)
    reduced_features = pca.transform(normalized_features)
    prediction = classifier.predict(reduced_features)
    return prediction

# Example usage
image_path = r"C:\Users\HP\Desktop\Image Recognitation\images\002_0087.jpg"
text_query = 'What is the color?'
response = get_model_response(image_path, text_query)
print(f"Model Response: {response}")

# Generate descriptions
def generate_description(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    outputs = model.generate(**inputs)
    description = processor.decode(outputs[0], skip_special_tokens=True)
    return description

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
description = generate_description(image_path)
print(f"Model Description: {description}")


Image preprocessing and feature extraction complete.




InvalidArgumentError: Graph execution error:

Detected at node sequential/embedding/embedding_lookup defined at (most recent call last):
  File "C:\ProgramData\anaconda3\envs\notebook\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\ProgramData\anaconda3\envs\notebook\lib\runpy.py", line 86, in _run_code

  File "C:\ProgramData\anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\ProgramData\anaconda3\envs\notebook\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\ProgramData\anaconda3\envs\notebook\lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\ProgramData\anaconda3\envs\notebook\lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\ProgramData\anaconda3\envs\notebook\lib\asyncio\base_events.py", line 603, in run_forever

  File "C:\ProgramData\anaconda3\envs\notebook\lib\asyncio\base_events.py", line 1909, in _run_once

  File "C:\Pro

In [49]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import torch
from torchvision import models
from PIL import Image
from transformers import BertTokenizer, BertModel, BlipProcessor, BlipForConditionalGeneration

# Setup paths
extract_folder = r"C:\Users\HP\Desktop\Image Recognitation\extracted_data"
processed_dir = r"C:\Users\HP\Desktop\Image Recognitation\processed_images"
features_dir = r"C:\Users\HP\Desktop\Image Recognitation\features"

# Create directories if not exist
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(features_dir, exist_ok=True)

# Image preprocessing function
def preprocess_image(img_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = cnn_model(img_tensor).squeeze().cpu().numpy()
    return features

# Define feature extraction for text
def extract_text_features(text):
    inputs = tokenizer(text, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Load models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_model = models.resnet18(pretrained=True).to(device)
cnn_model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

# Process images and extract features
for subdir in os.listdir(extract_folder):
    subdir_path = os.path.join(extract_folder, subdir)
    if os.path.isdir(subdir_path):
        for img_name in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_name)
            if os.path.isfile(img_path):
                try:
                    features = preprocess_image(img_path)
                    np.save(os.path.join(features_dir, img_name.replace('.jpg', '.npy')), features)
                except Exception as e:
                    print(f"Failed to process {img_name}: {e}")

print("Image preprocessing and feature extraction complete.")


C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: http

Image preprocessing and feature extraction complete.


In [ ]:

# Load question-response data
df = pd.read_csv(r'C:\Users\HP\Desktop\SIH\questions_responses.csv')
queries = df['Question'].values

# Tokenization and padding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(queries)
sequences = tokenizer.texts_to_sequences(queries)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Normalize sequences
def normalize_sequences(sequences):
    sequences_np = np.array(sequences, dtype=float)
    scaler = StandardScaler()
    return scaler.fit_transform(sequences_np)

normalized_sequences = normalize_sequences(padded_sequences)

# Define and train the LSTM model
embedding_dim = 128
lstm_units = 64
num_classes = 10
model = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_length),
    LSTM(lstm_units, return_sequences=False),
    Dense(lstm_units, activation='relu')
])
model.compile(optimizer='adam', loss='mean_squared_error')


In [55]:

# Assuming labels are defined and are available
# labels = np.array([...]) # Define your labels
# model.fit(normalized_sequences, labels, epochs=10, batch_size=32)

# Combine features for classification
# Replace placeholder with actual image features
image_features = np.array([preprocess_image(img_path) for img_path in os.listdir(features_dir) if img_path.endswith('.jpg')])
text_features = model.predict(normalized_sequences)
combined_features = np.concatenate((image_features, text_features), axis=1)

# Normalize and reduce dimensions
scaler = StandardScaler()
X_normalized = scaler.fit_transform(combined_features)
pca = PCA(n_components=100)
X_reduced = pca.fit_transform(X_normalized)

# Train and evaluate the classifier
labels = np.random.randint(0, num_classes, combined_features.shape[0])  # Placeholder
X_train, X_test, y_train, y_test = train_test_split(X_reduced, labels, test_size=0.3, random_state=42)
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy:.2f}")



InvalidArgumentError: Graph execution error:

Detected at node sequential_2/embedding_2/embedding_lookup defined at (most recent call last):
  File "C:\ProgramData\anaconda3\envs\notebook\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\ProgramData\anaconda3\envs\notebook\lib\runpy.py", line 86, in _run_code

  File "C:\ProgramData\anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\ProgramData\anaconda3\envs\notebook\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\ProgramData\anaconda3\envs\notebook\lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\ProgramData\anaconda3\envs\notebook\lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\ProgramData\anaconda3\envs\notebook\lib\asyncio\base_events.py", line 603, in run_forever

  File "C:\ProgramData\anaconda3\envs\notebook\lib\asyncio\base_events.py", line 1909, in _run_once

  File "C:

In [56]:
# Define feature extraction and response generation
def get_model_response(image_path, text_query):
    image_features = preprocess_image(image_path).reshape(1, -1)
    text_features = extract_text_features(text_query).reshape(1, -1)
    combined_features = np.concatenate((image_features, text_features), axis=1)
    normalized_features = scaler.transform(combined_features)
    reduced_features = pca.transform(normalized_features)
    prediction = classifier.predict(reduced_features)
    return prediction

# Example usage
image_path = r"C:\Users\HP\Desktop\Image Recognitation\images\002_0087.jpg"
text_query = 'What is the color?'
response = get_model_response(image_path, text_query)
print(f"Model Response: {response}")

# Generate descriptions
def generate_description(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    description = processor.decode(outputs[0], skip_special_tokens=True)
    return description

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
description = generate_description(image_path)
print(f"Model Description: {description}")


TypeError: 'Tokenizer' object is not callable